In [ ]:
import pandas as pd

%config InlineBackend.figure_format='retina'
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

rc('figure',figsize=(20,8))
rc('font',size=12)


In [ ]:
obj = pd.read_pickle('r_sigr.pkl')
obj.keys()
for k in obj.keys():
    print(k, np.shape(obj[k]))
r = obj['r']
sigr = obj['sigr']
nsub = obj['nsub']
correlation_length = obj['correlation_length']

allnsub = np.zeros_like(r)
allcorr = np.zeros_like(r)
for i in range(len(nsub)):
    allnsub[:,i,:] = nsub[i]
    
for i in range(len(correlation_length)):
    allcorr[i,:,:] = correlation_length[i]
    
# dd = {'r': np.ravel(r),
#      'sigr': np.ravel(sigr),
#      'nsub}


In [ ]:
mm = np.zeros((len(correlation_length), len(nsub)))
ss = np.zeros((len(correlation_length), len(nsub)))

subplot(1,2,1)
for k in range(len(correlation_length)):
    mm[k,:] = np.mean(r[k,:,:], axis=1)
    ss[k,:] = np.std(r[k,:,:], axis=1)
    errorbar(nsub, mm[k,:], yerr=ss[k,:], fmt='o', 
             label='Corr Length={}'.format(correlation_length[k]))
ylabel('Reconstructed r')
xlabel('Nsub')
legend()


In [ ]:
### Now reorder
sh = np.shape(r)
print(sh)

myrvalues = np.zeros((sh[2]*sh[0], sh[1]))
mycorrl = np.zeros(sh[2]*sh[0])
for i in range(sh[2]):
    for k in range(sh[0]):
        index = i*sh[0]+k
        myrvalues[index,:] = r[k,:,i]
        mycorrl[index] = correlation_length[k]

ok = (np.min(myrvalues, axis=1) > 0 ) & (np.min(myrvalues, axis=1) < 0.05 )
myrvalues = myrvalues[ok,:]
mycorrl = mycorrl[ok]

In [ ]:
myrvalues_normed = (myrvalues.T / myrvalues[:,0]).T

subplot(1,2,1)
for i in range(len(correlation_length)):
    this = mycorrl == correlation_length[i]
    errorbar(nsub, np.mean(myrvalues_normed[this,:], axis=0), 
             yerr = np.std(myrvalues_normed[this,:], axis=0)/np.sqrt(np.sum(this)), fmt='o', label='Corr Length = {}'.format(correlation_length[i]))
legend()

In [ ]:
dd = {'ns=3':myrvalues_normed[:,1],
     'ns=5':myrvalues_normed[:,2],
     'ns=8':myrvalues_normed[:,3],
     'CorrLength': mycorrl}
DataMathias = pd.DataFrame(dd)
DataMathias


In [ ]:

import seaborn as sn

target_column = "CorrLength"
columns = ["ns=3", "ns=5", "ns=8"]
_ = sn.pairplot(
    data=DataMathias,
    vars=columns,
    hue=target_column,
    plot_kws={"alpha": 0.2},
    height=2,
)

In [ ]:
data = DataMathias[['ns=3', 'ns=5', 'ns=8']]
target = DataMathias['CorrLength']
target.value_counts()

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate

#model = make_pipeline(StandardScaler(), LogisticRegression())
model = make_pipeline(StandardScaler(), LinearRegression())
#model = make_pipeline(LogisticRegression())
#model = make_pipeline(StandardScaler(), KNeighborsClassifier())

cv_result = cross_validate(model, data, target, cv=5, 
                           return_train_score=True)
cv_result = pd.DataFrame(cv_result)
print(cv_result)
print()
print(cv_result.mean())


In [ ]:
model.fit(data, target)
plot(target.values, model.predict(data), 'ro')

In [ ]:
import sys
sys.path.append("/Users/hamilton/Python/GitQubic/qubic/qubic/scripts/DiversJC/CalibSalta")
import fitting as fit


subplot(1,2,1)
tgu = np.sort(target.unique())
for i in range(len(tgu)):
    ok = target.values == tgu[i]
    print(tgu[i])
    fit.myhist(model.predict(data[ok]), range=[10,20], bins=100, alpha=0.5, label='Target = {}: '.format(tgu[i]))
legend()
xlabel('Reconstructed Correlation Length (training sample)')
